In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
data = pd.read_csv('../input/techuklon-int20h/train.csv')

data = data.groupby('Id').agg(lambda x: list(x))
columns = data.columns
for col in tqdm(data.columns):
    data[col] = data[col].apply(lambda x: np.array(x))
    data[col] = data[col].apply(lambda x: np.nan_to_num(x, np.nanmedian(x)) if not pd.isna(np.nanmedian(x)) else [np.nan]*4 )
    if col != 'target':
        for i in range(4):
            data[f'{col}_{i}'] = data[col].apply(lambda x: x[i])
        data = data.drop(columns = [col])
    else:
        data['target'] = data['target'].apply(lambda x: np.max(x))
data.drop(columns=['Week_0',"Week_1","Week_2","Week_3"], inplace=True)

In [ ]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
test_data = pd.read_csv('../input/techuklon-int20h/test.csv')

test_data = test_data.groupby('Id').agg(lambda x: list(x))
columns = test_data.columns
for col in tqdm(test_data.columns):
    test_data[col] = test_data[col].apply(lambda x: np.array(x))
    test_data[col] = test_data[col].apply(lambda x: np.nan_to_num(x, np.nanmedian(x)) if not pd.isna(np.nanmedian(x)) else [np.nan]*4 )
    if col != 'target':
        for i in range(4):
            test_data[f'{col}_{i}'] = test_data[col].apply(lambda x: x[i])
        test_data = test_data.drop(columns = [col])
    #else:
    #    #test_data['target'] = test_data['target'].apply(lambda x: np.max(x))
test_data.drop(columns=['Week_0',"Week_1","Week_2","Week_3"], inplace=True)

In [3]:
!pip install pycaret

In [5]:
from pycaret.classification import *
clf1 = setup(data, target = 'target', data_split_stratify = True) 

In [6]:
top3 = compare_models(n_select = 3)

In [29]:
model = tune_model(top3[0], optimize = 'AUC')

In [ ]:
#tuned_top3 = [tune_model(i, optimize = 'AUC') for i in top3]

In [30]:
res1 = predict_model(model, data = test_data, raw_score = True, round = 6).reset_index()[['Id','Score_1.0']].rename(columns = {'Score_1.0':'Predicted'})
#res2 = predict_model(top3[1], data = test_data, raw_score = True, round = 6).reset_index()[['Id','Score_1.0']].rename(columns = {'Score_1.0':'Predicted'})
#res3 = predict_model(top3[2], data = test_data, raw_score = True, round = 6).reset_index()[['Id','Score_1.0']].rename(columns = {'Score_1.0':'Predicted'})

In [32]:
#res1['Predicted'] = (res1['Predicted'] + res2['Predicted'] + res3['Predicted'])/3
res1.to_csv('final.csv', index =False)

In [ ]:
#blender = blend_models(estimator_list = top3) 